### Import Tensorflow and Numpy packages

In [25]:
import tensorflow as tf
import numpy as np
print("Tensorflow version: ", tf.__version__)
print("Numpy version     : ", np.__version__)

Tensorflow version:  1.10.0
Numpy version     :  1.15.2


### Download MNIST digit dataset
The MNIST database of handwritten digits, available from this page, has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='mnist.npz')

##### Process Data - Reshape mnist dataset for input node.

In [3]:
x_train = x_train.reshape([-1, 784])
x_train = x_train.astype(float)
y_train = y_train.astype(np.int32)

x_test = x_test.reshape([-1, 784])
x_test = x_test.astype(float)
y_test = y_test.astype(np.int32)

##### Check the data

In [4]:
# 60000 data samples
print("Train Samples: {}".format(x_train.shape))
print("Train Outputs: {}".format(y_train.shape))

# Sample size 28x28
# 10000 test samples
print("Eval Samples: {}".format(x_test.shape))
print("Eval Outputs: {}".format(y_test.shape))

Train Samples: (60000, 784)
Train Outputs: (60000,)
Eval Samples: (10000, 784)
Eval Outputs: (10000,)


### Write a model function
###### Model function runs in 3 different modes: Train, Evaluate and Test.
To implement a typical model function, you must do the following:
  - Define the model.
  - Specify additional calculations for each of the three different modes:
      - Predict
      - Evaluate
      - Train

In [5]:
def cnn_model_fn(features, labels, mode):
    
    input_layer = features["x"]
    fc1 = tf.layers.dense(input_layer, 256, activation=tf.nn.relu, name="Hidden-1")
    fc2 = tf.layers.dense(fc1, 128, activation=tf.nn.relu, name="Hidden-2")
    dropout = tf.layers.dropout(inputs=fc2, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1, name="classes_op"),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    # Compute evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels, predictions= predictions["classes"], name='acc_op')
    metrics = {'accuracy': accuracy}

    # merging and saving them every 100 steps by default for Tensorboard
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    tf.summary.scalar('loss', loss)

    internal_logging_hook = tf.train.LoggingTensorHook({"1_loss": loss, "2_accuracy": accuracy[1]}, every_n_iter=10 )

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op, training_hooks = [internal_logging_hook])

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


##### Create the Estimator

In [6]:
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                          model_dir="./checkpoints/log2jupyter",
                                          config=tf.estimator.RunConfig(log_step_count_steps=50,
                                                                        save_summary_steps=50),)


INFO:tensorflow:Using config: {'_model_dir': './checkpoints/log2jupyter', '_tf_random_seed': None, '_save_summary_steps': 50, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 50, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F4B410C908>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### Prepare input function for Training

In [7]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": x_train},
                                                    y=y_train,
                                                    batch_size=100,
                                                    num_epochs=None,
                                                    shuffle=True)


##### Train .. Train .. Train ..

In [19]:
# Train the model with external logging hook
mnist_classifier.train(input_fn=train_input_fn, steps=20000)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoints/log2jupyter\model.ckpt-45000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 45000 into ./checkpoints/log2jupyter\model.ckpt.
INFO:tensorflow:loss = 0.20390909910202026, step = 45001
INFO:tensorflow:1_loss = 0.20390909910202026, 2_accuracy = 0.93
INFO:tensorflow:1_loss = 0.20699363946914673, 2_accuracy = 0.93 (0.212 sec)
INFO:tensorflow:1_loss = 0.15673792362213135, 2_accuracy = 0.93666667 (0.062 sec)
INFO:tensorflow:1_loss = 0.1216132789850235, 2_accuracy = 0.9425 (0.069 sec)
INFO:tensorflow:1_loss = 0.09973914921283722, 2_accuracy = 0.946 (0.067 sec)
INFO:tensorflow:global_step/sec: 97.4592
INFO:tensorflow:loss = 0.17483079433441162, step = 45051 (0.515 sec)
INFO:tensorflow:1_loss = 0.17483079433441162, 2

INFO:tensorflow:1_loss = 0.08533430099487305, 2_accuracy = 0.9498701 (0.048 sec)
INFO:tensorflow:1_loss = 0.16885675489902496, 2_accuracy = 0.94948715 (0.063 sec)
INFO:tensorflow:1_loss = 0.14092959463596344, 2_accuracy = 0.94962025 (0.062 sec)
INFO:tensorflow:1_loss = 0.08711527287960052, 2_accuracy = 0.95 (0.047 sec)
INFO:tensorflow:global_step/sec: 183.871
INFO:tensorflow:loss = 0.06838176399469376, step = 45801 (0.285 sec)
INFO:tensorflow:1_loss = 0.06838176399469376, 2_accuracy = 0.9504938 (0.062 sec)
INFO:tensorflow:1_loss = 0.14148545265197754, 2_accuracy = 0.95036584 (0.063 sec)
INFO:tensorflow:1_loss = 0.09405551850795746, 2_accuracy = 0.95036143 (0.062 sec)
INFO:tensorflow:1_loss = 0.14630350470542908, 2_accuracy = 0.9502381 (0.062 sec)
INFO:tensorflow:1_loss = 0.03141925111413002, 2_accuracy = 0.9507059 (0.047 sec)
INFO:tensorflow:global_step/sec: 159.997
INFO:tensorflow:loss = 0.1564546376466751, step = 45851 (0.297 sec)
INFO:tensorflow:1_loss = 0.1564546376466751, 2_accura

INFO:tensorflow:1_loss = 0.11241785436868668, 2_accuracy = 0.95127386 (0.063 sec)
INFO:tensorflow:1_loss = 0.14983247220516205, 2_accuracy = 0.9512025 (0.051 sec)
INFO:tensorflow:1_loss = 0.1002308651804924, 2_accuracy = 0.95132077 (0.047 sec)
INFO:tensorflow:1_loss = 0.08316780626773834, 2_accuracy = 0.951375 (0.063 sec)
INFO:tensorflow:global_step/sec: 181.99
INFO:tensorflow:loss = 0.1224336251616478, step = 46601 (0.272 sec)
INFO:tensorflow:1_loss = 0.1224336251616478, 2_accuracy = 0.9514907 (0.062 sec)
INFO:tensorflow:1_loss = 0.14907410740852356, 2_accuracy = 0.9514198 (0.047 sec)
INFO:tensorflow:1_loss = 0.08790673315525055, 2_accuracy = 0.9514724 (0.047 sec)
INFO:tensorflow:1_loss = 0.048369769006967545, 2_accuracy = 0.9516463 (0.063 sec)
INFO:tensorflow:1_loss = 0.15870901942253113, 2_accuracy = 0.95151514 (0.047 sec)
INFO:tensorflow:global_step/sec: 177.773
INFO:tensorflow:loss = 0.17289943993091583, step = 46651 (0.281 sec)
INFO:tensorflow:1_loss = 0.17289943993091583, 2_accu

INFO:tensorflow:1_loss = 0.1420937329530716, 2_accuracy = 0.95181435 (0.047 sec)
INFO:tensorflow:1_loss = 0.0893402099609375, 2_accuracy = 0.95184875 (0.062 sec)
INFO:tensorflow:1_loss = 0.04529967159032822, 2_accuracy = 0.95200837 (0.061 sec)
INFO:tensorflow:1_loss = 0.20411384105682373, 2_accuracy = 0.95195836 (0.055 sec)
INFO:tensorflow:global_step/sec: 178.627
INFO:tensorflow:loss = 0.1865547150373459, step = 47401 (0.283 sec)
INFO:tensorflow:1_loss = 0.1865547150373459, 2_accuracy = 0.95182574 (0.060 sec)
INFO:tensorflow:1_loss = 0.24202826619148254, 2_accuracy = 0.95173556 (0.043 sec)
INFO:tensorflow:1_loss = 0.1454268842935562, 2_accuracy = 0.95176953 (0.062 sec)
INFO:tensorflow:1_loss = 0.16426900029182434, 2_accuracy = 0.9516803 (0.047 sec)
INFO:tensorflow:1_loss = 0.353544145822525, 2_accuracy = 0.95159185 (0.062 sec)
INFO:tensorflow:global_step/sec: 187.363
INFO:tensorflow:loss = 0.13592280447483063, step = 47451 (0.264 sec)
INFO:tensorflow:1_loss = 0.13592280447483063, 2_ac

INFO:tensorflow:1_loss = 0.08480454236268997, 2_accuracy = 0.9530915 (0.063 sec)
INFO:tensorflow:1_loss = 0.11543605476617813, 2_accuracy = 0.9530503 (0.093 sec)
INFO:tensorflow:1_loss = 0.19591359794139862, 2_accuracy = 0.9529154 (0.069 sec)
INFO:tensorflow:1_loss = 0.1449328362941742, 2_accuracy = 0.952875 (0.066 sec)
INFO:tensorflow:global_step/sec: 143.785
INFO:tensorflow:loss = 0.08969138562679291, step = 48201 (0.351 sec)
INFO:tensorflow:1_loss = 0.08969138562679291, 2_accuracy = 0.9529595 (0.062 sec)
INFO:tensorflow:1_loss = 0.03962898254394531, 2_accuracy = 0.9530745 (0.051 sec)
INFO:tensorflow:1_loss = 0.05124153196811676, 2_accuracy = 0.9531269 (0.117 sec)
INFO:tensorflow:1_loss = 0.09249284863471985, 2_accuracy = 0.9532099 (0.133 sec)
INFO:tensorflow:1_loss = 0.046825189143419266, 2_accuracy = 0.95332307 (0.343 sec)
INFO:tensorflow:global_step/sec: 70.9753
INFO:tensorflow:loss = 0.1387939453125, step = 48251 (0.717 sec)
INFO:tensorflow:1_loss = 0.1387939453125, 2_accuracy = 

...

INFO:tensorflow:1_loss = 0.09594222903251648, 2_accuracy = 0.95900416 (0.077 sec)
INFO:tensorflow:1_loss = 0.06372669339179993, 2_accuracy = 0.95902264 (0.055 sec)
INFO:tensorflow:1_loss = 0.1314864456653595, 2_accuracy = 0.9590173 (0.071 sec)
INFO:tensorflow:1_loss = 0.2073826789855957, 2_accuracy = 0.9590119 (0.061 sec)
INFO:tensorflow:global_step/sec: 150.252
INFO:tensorflow:loss = 0.045753706246614456, step = 61801 (0.334 sec)
INFO:tensorflow:1_loss = 0.045753706246614456, 2_accuracy = 0.95903033 (0.071 sec)
INFO:tensorflow:1_loss = 0.060264673084020615, 2_accuracy = 0.9590369 (0.069 sec)
INFO:tensorflow:1_loss = 0.13796046376228333, 2_accuracy = 0.95902556 (0.047 sec)
INFO:tensorflow:1_loss = 0.13449031114578247, 2_accuracy = 0.95902616 (0.066 sec)
INFO:tensorflow:1_loss = 0.04638095572590828, 2_accuracy = 0.95903856 (0.067 sec)
INFO:tensorflow:global_step/sec: 156.771
INFO:tensorflow:loss = 0.09720182418823242, step = 61851 (0.315 sec)
INFO:tensorflow:1_loss = 0.09720182418823242

INFO:tensorflow:1_loss = 0.14211544394493103, 2_accuracy = 0.9591975 (0.073 sec)
INFO:tensorflow:1_loss = 0.22523371875286102, 2_accuracy = 0.95917517 (0.070 sec)
INFO:tensorflow:1_loss = 0.0909910500049591, 2_accuracy = 0.95918137 (0.078 sec)
INFO:tensorflow:1_loss = 0.0368710495531559, 2_accuracy = 0.95919317 (0.064 sec)
INFO:tensorflow:global_step/sec: 142.602
INFO:tensorflow:loss = 0.06237047538161278, step = 62601 (0.348 sec)
INFO:tensorflow:1_loss = 0.06237047538161278, 2_accuracy = 0.9592107 (0.067 sec)
INFO:tensorflow:1_loss = 0.04365520924329758, 2_accuracy = 0.95922816 (0.074 sec)
INFO:tensorflow:1_loss = 0.10405875742435455, 2_accuracy = 0.9592229 (0.069 sec)
INFO:tensorflow:1_loss = 0.0795685425400734, 2_accuracy = 0.95922905 (0.071 sec)
INFO:tensorflow:1_loss = 0.03954598307609558, 2_accuracy = 0.95924646 (0.061 sec)
INFO:tensorflow:global_step/sec: 138.164
INFO:tensorflow:loss = 0.05525840073823929, step = 62651 (0.365 sec)
INFO:tensorflow:1_loss = 0.05525840073823929, 2_

INFO:tensorflow:1_loss = 0.11435096710920334, 2_accuracy = 0.9594012 (0.078 sec)
INFO:tensorflow:1_loss = 0.10039786994457245, 2_accuracy = 0.959407 (0.056 sec)
INFO:tensorflow:1_loss = 0.12959030270576477, 2_accuracy = 0.95940185 (0.068 sec)
INFO:tensorflow:1_loss = 0.0302286259829998, 2_accuracy = 0.9594185 (0.066 sec)
INFO:tensorflow:global_step/sec: 137.411
INFO:tensorflow:loss = 0.05140402540564537, step = 63401 (0.365 sec)
INFO:tensorflow:1_loss = 0.05140402540564537, 2_accuracy = 0.9594297 (0.098 sec)
INFO:tensorflow:1_loss = 0.05354120582342148, 2_accuracy = 0.9594517 (0.111 sec)
INFO:tensorflow:1_loss = 0.13980650901794434, 2_accuracy = 0.95944655 (0.121 sec)
INFO:tensorflow:1_loss = 0.0738959014415741, 2_accuracy = 0.95944685 (0.119 sec)
INFO:tensorflow:1_loss = 0.03621441125869751, 2_accuracy = 0.9594634 (0.083 sec)
INFO:tensorflow:global_step/sec: 90.4845
INFO:tensorflow:loss = 0.07892098277807236, step = 63451 (0.553 sec)
INFO:tensorflow:1_loss = 0.07892098277807236, 2_acc

INFO:tensorflow:1_loss = 0.07938386499881744, 2_accuracy = 0.9596505 (0.087 sec)
INFO:tensorflow:1_loss = 0.12893466651439667, 2_accuracy = 0.9596507 (0.061 sec)
INFO:tensorflow:1_loss = 0.09095275402069092, 2_accuracy = 0.9596509 (0.064 sec)
INFO:tensorflow:1_loss = 0.09734054654836655, 2_accuracy = 0.95965624 (0.051 sec)
INFO:tensorflow:global_step/sec: 148.745
INFO:tensorflow:loss = 0.13055628538131714, step = 64201 (0.332 sec)
INFO:tensorflow:1_loss = 0.13055628538131714, 2_accuracy = 0.9596564 (0.081 sec)
INFO:tensorflow:1_loss = 0.10692761838436127, 2_accuracy = 0.9596566 (0.068 sec)
INFO:tensorflow:1_loss = 0.07977583259344101, 2_accuracy = 0.9596568 (0.050 sec)
INFO:tensorflow:1_loss = 0.18924053013324738, 2_accuracy = 0.9596414 (0.067 sec)
INFO:tensorflow:1_loss = 0.19020873308181763, 2_accuracy = 0.9596208 (0.068 sec)
INFO:tensorflow:global_step/sec: 146.149
INFO:tensorflow:loss = 0.1481272131204605, step = 64251 (0.344 sec)
INFO:tensorflow:1_loss = 0.1481272131204605, 2_accu

INFO:tensorflow:1_loss = 0.11696632951498032, 2_accuracy = 0.9597797 (0.047 sec)
INFO:tensorflow:1_loss = 0.08436376601457596, 2_accuracy = 0.9597848 (0.063 sec)
INFO:tensorflow:1_loss = 0.09491261094808578, 2_accuracy = 0.9597949 (0.047 sec)
INFO:tensorflow:1_loss = 0.050749652087688446, 2_accuracy = 0.959805 (0.064 sec)
INFO:tensorflow:Saving checkpoints for 65000 into ./checkpoints/log2jupyter\model.ckpt.
INFO:tensorflow:Loss for final step: 0.044557083398103714.


##### Prepare input function for Evaluation

In [29]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": x_test},
                                                   y=y_test,
                                                   num_epochs=1,
                                                   shuffle=False)

##### Evaluate

In [30]:
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-04-14:00:16
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoints/log2jupyter\model.ckpt-65000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-04-14:00:16
INFO:tensorflow:Saving dict for global step 65000: accuracy = 0.966, global_step = 65000, loss = 0.18617551
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 65000: ./checkpoints/log2jupyter\model.ckpt-65000


### Final result

In [31]:
print(eval_results)

{'accuracy': 0.966, 'loss': 0.18617551, 'global_step': 65000}
